In [55]:
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

# #generate only year over year time frames from a list of years
# def generate_time_frames(years):
#     time_frames = []
#     for i in range(len(years)-1):
#         time_frames.append(f"{years[i]}-{years[i+1]}")
#     return time_frames
#change calcs
# def calculate_changes(df, columns, time_frames, years):
#     for column in columns:
#         for time_frame in time_frames:
#             start_year, end_year = years[time_frames.index(time_frame)], years[time_frames.index(time_frame) + 1]
#             df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, end_year, 'None')], df[(column, start_year, 'None')])
#             df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, end_year, 'None')] - df[(column, start_year, 'None')])

#     return df

def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df

#functions
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

In [56]:
import pandas as pd
import numpy as np
import requests
from collections import deque
from functools import reduce
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
import sqlite3 as sq

# This notebook outlines the download and formatting process for the HUD residential building permits dataset for counties and places in the GNRC operating region.

There are two figures needed from this information, one is a history of all permits; historical data for all single and multifamily (we can add these for total permits and it's interesting to see both).

Go to this page: https://socds.huduser.gov/permits/  

##### Download 2  
+ Under "Main Criteria" select "States and Counties"  
+ Under "Periodicity" select "Annual"  
+ Under "Select State(s)" select "Tennessee", then select "Show Counties and Jurisdictions for Selected States"  
+ Under "Select Counties", highlight all counties in the GNRC region, check the box for "county total", and show/select all under "Select Permitting Jurisdictions" 
+ Under "Select Series", select all options  
+ Select the most recent year  
+ "Get Data"  
+ Download the file and save as "HUD_DisaggregatedBuildingPermits_xxxx.csv"  in "Data Downloads"

In [57]:
#import high level permit data for all years and disaggregated permits for most recent year
data = pd.read_csv('../Data Downloads/HUD_DisaggregatedBuildingPermits_2023.csv')
latestyear = '2023'
data = data.drop(columns = 'Year')

In [58]:
#examine
data.tail(10)

,Location,Series,Series Code,Permits
410,WILLIAMSON COUNTY UNINCORPORATED AREA,Units in All Multi-Family Structures,3,0
411,WILLIAMSON COUNTY UNINCORPORATED AREA,Units in 2-unit Multi-Family Structures,4,0
412,WILLIAMSON COUNTY UNINCORPORATED AREA,Units in 3- and 4-unit Multi-Family Structures,5,0
413,WILLIAMSON COUNTY UNINCORPORATED AREA,Units in 5+ Unit Multi-Family Structures,6,0
414,WILSON COUNTY UNINCORPORATED AREA,Total Units,1,368
415,WILSON COUNTY UNINCORPORATED AREA,Units in Single-Family Structures,2,368
416,WILSON COUNTY UNINCORPORATED AREA,Units in All Multi-Family Structures,3,0
417,WILSON COUNTY UNINCORPORATED AREA,Units in 2-unit Multi-Family Structures,4,0
418,WILSON COUNTY UNINCORPORATED AREA,Units in 3- and 4-unit Multi-Family Structures,5,0
419,WILSON COUNTY UNINCORPORATED AREA,Units in 5+ Unit Multi-Family Structures,6,0


In [59]:
#If you examine the data download in excel, you can see that the headers repeat every few rows, delete these by indexing 
data = data.loc[data['Location'] != 'Location']

In [60]:
#strip extra spaces
data['Location'] = data['Location'].str.strip()

We have three different target geographies that are downloaded: counties, unincorporated areas, and places. We are only taking geographies within the GNRC 14 county operating region. You can see from the download that all counties contain the string "County", and that unincorporated areas contain the string "COUNTY".

In [61]:
counties = data[data['Location'].str.contains('County')]

In [62]:
#append the state for use later
counties['Location'] = counties['Location'] + ", Tennessee"

Counties are ready to go, now for Unincorporated Areas

In [63]:
#unincorporated areas were in all capital letters
unincorporated = data[data['Location'].str.contains('COUNTY')]

In [64]:
region = ['CHEATHAM COUNTY UNINCORPORATED AREA', 'DAVIDSON COUNTY UNINCORPORATED AREA', 'DICKSON COUNTY UNINCORPORATED AREA', 
          'HOUSTON COUNTY UNINCORPORATED AREA', 'HUMPHREYS COUNTY UNINCORPORATED AREA', 'MAURY COUNTY UNINCORPORATED AREA', 
          'MONTGOMERY COUNTY UNINCORPORATED AREA', 'ROBERTSON COUNTY UNINCORPORATED AREA', 'RUTHERFORD COUNTY UNINCORPORATED AREA', 
          'STEWART COUNTY UNINCORPORATED AREA', 'SUMNER COUNTY UNINCORPORATED AREA', 'TROUSDALE COUNTY UNINCORPORATED AREA', 
          'WILLIAMSON COUNTY UNINCORPORATED AREA', 'WILSON COUNTY UNINCORPORATED AREA']

#Davidson, Houston, Humphreys nor Stewart have unincorporated lines

In [65]:
unincdict = {'CHEATHAM COUNTY UNINCORPORATED AREA': 'Cheatham Unincorporated', 'DICKSON COUNTY UNINCORPORATED AREA': "Dickson Unincorporated", 
             'MAURY COUNTY UNINCORPORATED AREA': 'Maury Unincorporated', 'MONTGOMERY COUNTY UNINCORPORATED AREA': 'Montgomery Unincorporated', 
             'ROBERTSON COUNTY UNINCORPORATED AREA': 'Robertson Unincorporated', 'RUTHERFORD COUNTY UNINCORPORATED AREA': 'Rutherford Unincorporated',
             'SUMNER COUNTY UNINCORPORATED AREA': 'Sumner Unincorporated', 'TROUSDALE COUNTY UNINCORPORATED AREA': 'Trousdale Unincorporated', 
             'WILLIAMSON COUNTY UNINCORPORATED AREA': 'Williamson Unincorporated', 'WILSON COUNTY UNINCORPORATED AREA': 'Wilson Unincorporated'}
unincorporated['Location'] = unincorporated['Location'].replace(unincdict)

Places are going to be more difficult, we're looking for the following: 
Adams city, Tennessee: Robertson  
Ashland City town, Tennessee: Cheatham  
Belle Meade city, Tennessee: Davidson  
Berry Hill city, Tennessee: Davidson  
Brentwood city, Tennessee: Williamson  
Burns town, Tennessee: Dickson  
Cedar Hill city, Tennessee: Robertson  
Charlotte town, Tennessee: Dickson  
Clarksville city, Tennessee: Montgomery  
Columbia city, Tennessee: Maury  
Coopertown town, Tennessee: Robertson  
Cross Plains city, Tennessee: Robertson  
Cumberland City town, Tennessee: Stewart  
Dickson city, Tennessee: Dickson  
Dover city, Tennessee: Stewart  
Eagleville city, Tennessee: Rutherford  
Erin city, Tennessee: Houston  
Fairview city, Tennessee: Williamson  
Forest Hills city, Tennessee: Davidson  
Franklin city, Tennessee: Williamson  
Gallatin city, Tennessee: Sumner  
Goodlettsville city, Tennessee: Davidson/Sumner  
Greenbrier town, Tennessee: Robertson  
Hendersonville city, Tennessee: Sumner  
Kingston Springs town, Tennessee: Cheatham  
La Vergne city, Tennessee: Rutherford  
Lafayette city, Tennessee: Macon  
Lebanon city, Tennessee: Wilson  
McEwen city, Tennessee: Humphreys  
Millersville city, Tennessee: Robertson/Sumner  
Mitchellville city, Tennessee: Sumner  
Mount Juliet city, Tennessee: Wilson  
Mount Pleasant city, Tennessee: Maury  
Murfreesboro city, Tennessee: Rutherford  
Nashville-Davidson metropolitan government (balance): Davidson  
New Johnsonville city, Tennessee: Humphreys  
Nolensville town, Tennessee: Williamson  
Oak Hill city, Tennessee: Davidson  
Pegram town, Tennessee: Cheatham  
Pleasant View city, Tennessee: Cheatham  
Portland city, Tennessee: Robertson/Sumner  
Ridgetop city, Tennessee: Davidson/Robertson  
Slayden town, Tennessee: Dickson  
Smyrna town, Tennessee: Rutherford  
Spring Hill city, Tennessee: Maury/Williamson  
Springfield city, Tennessee: Robertson  
Tennessee Ridge town, Tennessee: Houston/Stewart  
Thompson's Station town, Tennessee: Williamson  
Vanleer town, Tennessee: Dickson  
Watertown city, Tennessee: Wilson  
Waverly city, Tennessee: Humphreys  
Westmoreland town, Tennessee: Sumner  
White Bluff town, Tennessee: Dickson  
White House city, Tennessee: Robertson/Sumner    

The downloaded data is in all caps, and doesn't have the ", Tennessee" of the "town.. city... etc.". There are 54 records. We can reverse index into the geographies that are *not* counties or unincorporated areas, reformat the capital letters, and then I'll make a list of only the first word of the place to run through and see how close we can get that way.


In [66]:
#for places filter out the counties and unincorporated areas
place = data[~data['Location'].str.contains('County')]
place = place[~place['Location'].str.contains('COUNTY')]

In [67]:
#this makes the first letter of each word uppercase and the reset lowercase
place['Location'] = place['Location'].str.title()

In [68]:
#I've gone through the excel document to see what these are named - hopefully it is consistent year to year so that this saves time.
places = ["Adams", "Ashland City Town", "Belle Meade", "Berry Hill", "Brentwood", "Burns Town",  "Charlotte Town", "Clarksville", "Columbia", "Coopertown Town", 
          "Cross Plains", "Cumberland City Town", "Dickson", "Dover", "Eagleville", "Erin", "Fairview", "Forest Hills", "Franklin", "Gallatin", "Goodlettsville", 
          "Greenbrier Town","Hendersonville", "Kingston Springs Town", "La Vergne", "Lafayette", "Lebanon", "Mcewen", "Millersville", "Mitchellville Town", 
          "Mount Juliet", "Mount Pleasant", "Murfreesboro", "Nashville-Davidson", "New Johnsonville", "Nolensville Town", "Oak Hill", "Pegram Town", "Pleasant View", 
          "Portland", "Ridgetop Town","Smyrna Town", "Spring Hill Town", "Springfield", "Tennessee Ridge Town", "Thompsons Station Town", "Watertown", 
          "Waverly", "Westmoreland Town", "White Bluff Town", "White House"]

In [69]:
#filter both df's with the list of places above
place = place.loc[place['Location'].isin(places)].reset_index(drop = True)

In [70]:
#transpose in order to rename
transp = place.set_index('Location').transpose()

In [71]:
transp.head()

Location,Adams,Adams,Adams,Adams,Adams,Adams,Ashland City Town,Ashland City Town,Ashland City Town,Ashland City Town,Ashland City Town,Ashland City Town,Belle Meade,Belle Meade,Belle Meade,Belle Meade,Belle Meade,Belle Meade,Berry Hill,Berry Hill,Berry Hill,Berry Hill,Berry Hill,Berry Hill,Brentwood,Brentwood,Brentwood,Brentwood,Brentwood,Brentwood,Burns Town,Burns Town,Burns Town,Burns Town,Burns Town,Burns Town,Charlotte Town,Charlotte Town,Charlotte Town,Charlotte Town,Charlotte Town,Charlotte Town,Clarksville,Clarksville,Clarksville,Clarksville,Clarksville,Clarksville,Columbia,Columbia,Columbia,Columbia,Columbia,Columbia,Coopertown Town,Coopertown Town,Coopertown Town,Coopertown Town,Coopertown Town,Coopertown Town,Cross Plains,Cross Plains,Cross Plains,Cross Plains,Cross Plains,Cross Plains,Dickson,Dickson,Dickson,Dickson,Dickson,Dickson,Eagleville,Eagleville,Eagleville,Eagleville,Eagleville,Eagleville,Fairview,Fairview,Fairview,Fairview,Fairview,Fairview,Forest Hills,Forest Hills,Forest Hills,Forest Hills,Forest Hills,Forest Hills,Franklin,Franklin,Franklin,Franklin,Franklin,Franklin,Gallatin,Gallatin,Gallatin,Gallatin,Gallatin,Gallatin,Goodlettsville,Goodlettsville,Goodlettsville,Goodlettsville,Goodlettsville,Goodlettsville,Greenbrier Town,Greenbrier Town,Greenbrier Town,Greenbrier Town,Greenbrier Town,Greenbrier Town,Hendersonville,Hendersonville,Hendersonville,Hendersonville,Hendersonville,Hendersonville,Kingston Springs Town,Kingston Springs Town,Kingston Springs Town,Kingston Springs Town,Kingston Springs Town,Kingston Springs Town,La Vergne,La Vergne,La Vergne,La Vergne,La Vergne,La Vergne,Lebanon,Lebanon,Lebanon,Lebanon,Lebanon,Lebanon,Millersville,Millersville,Millersville,Millersville,Millersville,Millersville,Mitchellville Town,Mitchellville Town,Mitchellville Town,Mitchellville Town,Mitchellville Town,Mitchellville Town,Mount Juliet,Mount Juliet,Mount Juliet,Mount Juliet,Mount Juliet,Mount Juliet,Mount Pleasant,Mount Pleasant,Mount Pleasant,Mount Pleasant,Mount Pleasant,Mount Pleasant,Murfreesboro,Murfreesboro,Murfreesboro,Murfreesboro,Murfreesboro,Murfreesboro,Nashville-Davidson,Nashville-Davidson,Nashville-Davidson,Nashville-Davidson,Nashville-Davidson,Nashville-Davidson,Nolensville Town,Nolensville Town,Nolensville Town,Nolensville Town,Nolensville Town,Nolensville Town,Oak Hill,Oak Hill,Oak Hill,Oak Hill,Oak Hill,Oak Hill,Pegram Town,Pegram Town,Pegram Town,Pegram Town,Pegram Town,Pegram Town,Pleasant View,Pleasant View,Pleasant View,Pleasant View,Pleasant View,Pleasant View,Portland,Portland,Portland,Portland,Portland,Portland,Ridgetop Town,Ridgetop Town,Ridgetop Town,Ridgetop Town,Ridgetop Town,Ridgetop Town,Smyrna Town,Smyrna Town,Smyrna Town,Smyrna Town,Smyrna Town,Smyrna Town,Spring Hill Town,Spring Hill Town,Spring Hill Town,Spring Hill Town,Spring Hill Town,Spring Hill Town,Springfield,Springfield,Springfield,Springfield,Springfield,Springfield,Thompsons Station Town,Thompsons Station Town,Thompsons Station Town,Thompsons Station Town,Thompsons Station Town,Thompsons Station Town,Watertown,Watertown,Watertown,Watertown,Watertown,Watertown,Westmoreland Town,Westmoreland Town,Westmoreland Town,Westmoreland Town,Westmoreland Town,Westmoreland Town,White Bluff Town,White Bluff Town,White Bluff Town,White Bluff Town,White Bluff Town,White Bluff Town,White House,White House,White House,White House,White House,White House
Series,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Units in 2-unit Multi-Family Structures,Units in 3- and 4-unit Multi-Family Structures,Units in 5+ Unit Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Units in 2-unit Multi-Family Structures,Units in 3- and 4-unit Multi-Family Structures,Units in 5+ Unit Multi-Family Structures,Total Units,Units in Single-Family Structures,Units in All Multi-Family Structures,Units in 2-unit Multi-Family Structures,Units in 3- and 4-unit Multi-Family Structu

In [72]:
#don't even try to use the custom module for this - this part is taking their weird names and making them into Census correct names
transp = transp.rename(columns = {"Adams": 'Adams city, Tennessee', "Ashland City Town":'Ashland City town, Tennessee',
                                  "Belle Meade": 'Belle Meade city, Tennessee', "Berry Hill": 'Berry Hill city, Tennessee', 
                                  "Brentwood": 'Brentwood city, Tennessee', "Burns Town": 'Burns town, Tennessee',  
                                  "Charlotte Town": 'Charlotte town, Tennessee',  "Clarksville": 'Clarksville city, Tennessee', 
                                  "Columbia": 'Columbia city, Tennessee', "Coopertown Town": 'Coopertown town, Tennessee', 
                                  "Cross Plains": 'Cross Plains city, Tennessee', "Cumberland City Town":'Cumberland City town, Tennessee', 
                                  "Dickson": 'Dickson city, Tennessee', "Dover": 'Dover city, Tennessee', 
                                  "Eagleville": 'Eagleville city, Tennessee', "Erin": 'Erin city, Tennessee', 
                                  "Fairview": 'Fairview city, Tennessee', "Forest Hills": 'Forest Hills city, Tennessee', 
                                  "Franklin": 'Franklin city, Tennessee', "Gallatin": 'Gallatin city, Tennessee', 
                                  "Goodlettsville": 'Goodlettsville city, Tennessee', "Greenbrier Town": 'Greenbrier town, Tennessee',
                                  "Hendersonville": 'Hendersonville city, Tennessee', "Kingston Springs Town": 'Kingston Springs town, Tennessee', 
                                  "La Vergne": 'La Vergne city, Tennessee', "Lafayette": 'La Vergne city, Tennessee', 
                                  "Lebanon": 'Lebanon city, Tennessee', "Mcewen": 'McEwen city, Tennessee', 
                                  "Millersville": 'Millersville city, Tennessee', "Mitchellville Town": 'Mitchellville city, Tennessee', 
                                  "Mount Juliet": 'Mount Juliet city, Tennessee', "Mount Pleasant": 'Mount Pleasant city, Tennessee', 
                                  "Murfreesboro": 'Murfreesboro city, Tennessee', "Nashville-Davidson": "Nashville-Davidson metropolitan government (balance)", 
                                  "New Johnsonville": 'New Johnsonville city, Tennessee', "Nolensville Town": 'Nolensville town, Tennessee', 
                                  "Oak Hill": 'Oak Hill city, Tennessee', "Pegram Town": 'Pegram town, Tennessee', 
                                  "Pleasant View": 'Pleasant View city, Tennessee', "Portland": 'Portland city, Tennessee', 
                                  "Ridgetop Town": 'Ridgetop city, Tennessee',"Smyrna Town": 'Smyrna town, Tennessee', 
                                  "Spring Hill Town": 'Spring Hill city, Tennessee', "Springfield": 'Springfield city, Tennessee', 
                                  "Tennessee Ridge Town": 'Tennessee Ridge town, Tennessee', "Thompsons Station Town": "Thompson's Station town, Tennessee",
                                  "Watertown": 'Watertown city, Tennessee', "Waverly": 'Waverly city, Tennessee', 
                                  "Westmoreland Town": 'Westmoreland town, Tennessee', "White Bluff Town": 'White Bluff town, Tennessee', 
                                  "White House": 'White House city, Tennessee'})

In [73]:
#transpose back and reset index without dropping
place = transp.transpose().reset_index()

In [74]:
#concatenate the annual time series data and the current data
dfs = [counties, unincorporated, place]
df = pd.concat(dfs)

In [75]:
#rename location to name and drop series code
df = df.rename(columns = {'Location':'NAME'})
df = df.drop(columns = 'Series Code')

In [76]:
#pivot these dataframes
df = df.pivot(index = ['NAME'], columns = 'Series', values = 'Permits')

In [77]:
df = df.reset_index(drop = False)

In [78]:
data = df.set_index(['NAME']).transpose()

In [79]:
data.head(2)

NAME,"Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Charlotte town, Tennessee","Cheatham County, Tennessee",Cheatham Unincorporated,"Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee",Dickson Unincorporated,"Dickson city, Tennessee","Eagleville city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Kingston Springs town, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","Maury County, Tennessee",Maury Unincorporated,"Millersville city, Tennessee","Mitchellville city, Tennessee","Montgomery County, Tennessee",Montgomery Unincorporated,"Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee",Nashville-Davidson metropolitan government (balance),"Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Robertson County, Tennessee",Robertson Unincorporated,"Rutherford County, Tennessee",Rutherford Unincorporated,"Smyrna town, Tennessee","Spring Hill city, Tennessee","Springfield city, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee",Sumner Unincorporated,"Thompson's Station town, Tennessee","Trousdale County, Tennessee",Trousdale Unincorporated,"Watertown city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee","Williamson County, Tennessee",Williamson Unincorporated,"Wilson County, Tennessee",Wilson Unincorporated
Series,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Total Units,0,73,4,5,104,26,0,285,123,2260,690,23,0,11225,412,227,62,11,120,0,306,1058,33,24,265,0,34,5,186,1233,1562,197,49,0,2784,524,273,22,1320,11158,121,25,7,77,37,2,939,142,3540,884,1139,653,89,10,1746,327,184,122,122,0,10,97,659,1432,597,1874,368
Units in 2-unit Multi-Family Structures,0,0,0,0,0,0,0,4,0,60,0,0,0,12,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,0,68,8,0,0,6,12,0,0,0,4,18,0,2,0,6,0,0,0,2,0,22,0,0,4,4,0,2,6,0,0,0,0,0


In [80]:
cols = data.columns
data[cols] = data[cols].astype(float)
GNRCCounties = [data[('Stewart County, Tennessee')],data[('Montgomery County, Tennessee')],
                data[('Houston County, Tennessee')],data[('Humphreys County, Tennessee')],
                data[('Dickson County, Tennessee')],data[('Cheatham County, Tennessee')],
                data[('Robertson County, Tennessee')],data[('Sumner County, Tennessee')],
                data[('Davidson County, Tennessee')],data[('Wilson County, Tennessee')],
                data[('Trousdale County, Tennessee')],data[('Williamson County, Tennessee')],
                data[('Rutherford County, Tennessee')]]
data[('GNRC')] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee')],data[('Montgomery County, Tennessee')],
                   data[('Houston County, Tennessee')],data[('Humphreys County, Tennessee')],
                   data[('Dickson County, Tennessee')],data[('Cheatham County, Tennessee')],
                   data[('Robertson County, Tennessee')],data[('Sumner County, Tennessee')],
                   data[('Davidson County, Tennessee')],data[('Wilson County, Tennessee')],
                   data[('Trousdale County, Tennessee')],data[('Williamson County, Tennessee')],
                   data[('Rutherford County, Tennessee')],data[('Maury County, Tennessee')]]
data[('GNRC Region')] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee')],data[('Sumner County, Tennessee')],
               data[('Davidson County, Tennessee')],data[('Wilson County, Tennessee')],
               data[('Williamson County, Tennessee')],data[('Rutherford County, Tennessee')],
               data[('Maury County, Tennessee')]]
data[('MPO')] = sum(MPOCounties)
data = data.transpose().reset_index()
df = data

In [81]:
df.head()

Series,NAME,Total Units,Units in 2-unit Multi-Family Structures,Units in 3- and 4-unit Multi-Family Structures,Units in 5+ Unit Multi-Family Structures,Units in All Multi-Family Structures,Units in Single-Family Structures
0,"Adams city, Tennessee",0.0,0.0,0.0,0.0,0.0,0.0
1,"Ashland City town, Tennessee",73.0,0.0,0.0,63.0,63.0,10.0
2,"Belle Meade city, Tennessee",4.0,0.0,0.0,0.0,0.0,4.0
3,"Berry Hill city, Tennessee",5.0,0.0,0.0,0.0,0.0,5.0
4,"Brentwood city, Tennessee",104.0,0.0,0.0,0.0,0.0,104.0


In [82]:
#rename
df = df.rename(columns = {'Units in 3- and 4-unit Multi-Family Structures': 'Units in 3 and 4 Unit Multi-Family Structures', 
                            'Units in 2-unit Multi-Family Structures': 'Units in 2 Unit Multi-Family Structures', 
                            'Total Units': 'Units in All Structures'})

In [83]:
#map the common name to the geoid dict in custom module
df['GEO_ID'] = df['NAME'].map(geotogeoid)
#set source
df['Source'] = 'HUD SOC Annual Data {}'.format(latestyear)
#this is a way to order the columns
df = df[['NAME', 'GEO_ID', 'Units in All Structures', 'Units in Single-Family Structures', 
           'Units in All Multi-Family Structures', 'Units in 2 Unit Multi-Family Structures', 
           'Units in 3 and 4 Unit Multi-Family Structures', 'Units in 5+ Unit Multi-Family Structures', 'Source']]

In [84]:
df['Source'] = 'HUD SOC 2023 Data'

In [85]:
df['NAME'].unique()

array(['Adams city, Tennessee', 'Ashland City town, Tennessee',
       'Belle Meade city, Tennessee', 'Berry Hill city, Tennessee',
       'Brentwood city, Tennessee', 'Burns town, Tennessee',
       'Charlotte town, Tennessee', 'Cheatham County, Tennessee',
       'Cheatham Unincorporated', 'Clarksville city, Tennessee',
       'Columbia city, Tennessee', 'Coopertown town, Tennessee',
       'Cross Plains city, Tennessee', 'Davidson County, Tennessee',
       'Dickson County, Tennessee', 'Dickson Unincorporated',
       'Dickson city, Tennessee', 'Eagleville city, Tennessee',
       'Fairview city, Tennessee', 'Forest Hills city, Tennessee',
       'Franklin city, Tennessee', 'Gallatin city, Tennessee',
       'Goodlettsville city, Tennessee', 'Greenbrier town, Tennessee',
       'Hendersonville city, Tennessee', 'Houston County, Tennessee',
       'Humphreys County, Tennessee', 'Kingston Springs town, Tennessee',
       'La Vergne city, Tennessee', 'Lebanon city, Tennessee',
       '

In [86]:
df.head(2)

Series,NAME,GEO_ID,Units in All Structures,Units in Single-Family Structures,Units in All Multi-Family Structures,Units in 2 Unit Multi-Family Structures,Units in 3 and 4 Unit Multi-Family Structures,Units in 5+ Unit Multi-Family Structures,Source
0,"Adams city, Tennessee",1600000US4700200,0.0,0.0,0.0,0.0,0.0,0.0,HUD SOC 2023 Data
1,"Ashland City town, Tennessee",1600000US4702180,73.0,10.0,63.0,0.0,0.0,63.0,HUD SOC 2023 Data


In [87]:
#export to the SQLite database
conn = sq.connect('../Outputs/HUD.db')
df.to_sql('AllPermitTypes_{}'.format(latestyear), conn, if_exists = 'replace', index = False)

70